En este pipeline se agregan las celdas 11.5 y 11.6. Se calculan precios promedios en base a propiedades cercanas, y tambien se utilizan datos de https://www.estadisticaciudad.gob.ar/eyc/?p=157955 que nos da precio promedio del m2 en Buenos Aires

In [6]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import joblib

# 1. Cargar datos
data = pd.read_csv('zonaprop_propiedades.csv')


In [7]:
# 2. Procesar tipo de vivienda
def obtener_clase(valor):
    if pd.isnull(valor):
        return None
    return valor.split('·')[0].strip()
data['vivienda'] = data['title'].apply(obtener_clase)


In [8]:
# 3. Procesar precio
ARS_TO_USD = 1 / 1200 # Tasa de conversión aproximada, ajustar según sea necesario
def convertir_precio(valor):
    if pd.isnull(valor):
        return None
    try:
        partes = valor.split()
        moneda = partes[0]
        numero_str = partes[1].replace('.', '').replace(',', '')
        numero = int(numero_str)
        if moneda == 'USD':
            return numero
        elif moneda == 'ARS':
            return int(numero * ARS_TO_USD)
        else:
            return None
    except:
        return None
data['price'] = data['rent_price'].apply(convertir_precio)


In [ ]:
import time
from geopy.geocoders import OpenCage

api_key = "2da76b0a220c4d239c9c251fc20bab83"
geolocator = OpenCage(api_key)

def geocodear(direccion):
    try:
        ubicacion = geolocator.geocode(direccion + ', Argentina')
        if ubicacion:
            print(f'{ubicacion.latitude}, {ubicacion.longitude}')
            return ubicacion.latitude, ubicacion.longitude
        else:
            return None, None
    except Exception as e:
        #print(f"Error geocodificando la dirección {direccion}: {e}")
        return None, None

data[['latitud', 'longitud']] = data['location'].apply(lambda x: pd.Series(geocodear(x)))
time.sleep(0.25)  # Esperar para evitar exceder el límite de solicitudes
data[['location', 'latitud', 'longitud']].head()

data_old = data.copy()

In [10]:
# 4. Procesar expensas
def convertir_expensas(valor):
    if pd.isnull(valor):
        return None
    if "No disponible" in valor:
        return None
    try:
        numero_str = valor.replace('Expensas $', '').strip().replace('.', '').replace(',', '')
        numero = int(numero_str)
        return numero
    except:
        return None
data['expenses'] = data['expenses_price'].apply(convertir_expensas)


In [11]:
# 5. Extraer variables numéricas
def extraer_numero_regex(valor):
    if pd.isnull(valor):
        return None
    try:
        match = re.search(r'(\d+(?:[.,]\d+)?)', valor)
        if match:
            numero_str = match.group(1).replace(',', '.')
            if '.' in numero_str:
                return float(numero_str)
            else:
                return int(numero_str)
        else:
            return None
    except:
        return None

columnas_mapeo = {
    'icon-stotal': 'm2_totales',
    'icon-scubierta': 'm2_cubiertos',
    'icon-ambiente': 'ambientes',
    'icon-bano': 'baños',
    'icon-cochera': 'cocheras',
    'icon-dormitorio': 'dormitorios'
}
for col_original, col_nueva in columnas_mapeo.items():
    data[col_nueva] = data[col_original].apply(extraer_numero_regex)


In [12]:
# 6. Procesar antigüedad
def convertir_antiguedad(valor):
    if pd.isnull(valor):
        return None
    if 'A estrenar' in valor:
        return 0
    try:
        match = re.search(r'\d+', valor)
        if match:
            return int(match.group(0))
        else:
            return None
    except:
        return None
data['antiguedad'] = data['icon-antiguedad'].apply(convertir_antiguedad)


In [13]:
# 7. Procesar features generales
data['general_features'] = data['general_features'].fillna('').astype(str)
def extract_plants(text):
    match = re.search(r'Cantidad plantas\s*:\s*(\d+|5 o más)', text)
    if match:
        if match.group(1) == '5 o más':
            return 5
        return int(match.group(1))
    return 1
def has_pool(text):
    return 'sí' if 'Pileta' in text else 'no'
def is_credit_compatible(text):
    return 'sí' if 'Apto profesional' in text else 'no'
data['Cantidad_plantas'] = data['general_features'].apply(extract_plants)
data['Pileta'] = data['general_features'].apply(has_pool)
data['Apto_credito'] = data['general_features'].apply(is_credit_compatible)


In [14]:
# 8. Seleccionar columnas relevantes

columns_to_keep = [
    'Cantidad_plantas', 'Pileta', 'Apto_credito', 'antiguedad', 'dormitorios',
    'cocheras', 'baños', 'ambientes', 'm2_totales', 'm2_cubiertos', 'expenses',
    'price', 'vivienda','latitud', 'longitud'
]
data = data[columns_to_keep]


In [15]:
# 9. Eliminar outliers extremos (3*IQR)
def get_extreme_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 3 * IQR
    upper_bound = Q3 + 3 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers
columns_numeric = ['price', 'm2_totales', 'm2_cubiertos', 'dormitorios', 'baños', 'antiguedad']
extreme_outliers = pd.DataFrame()
for column in columns_numeric:
    outliers = get_extreme_outliers(data, column)
    extreme_outliers = pd.concat([extreme_outliers, outliers])
extreme_outliers = extreme_outliers.drop_duplicates()
data = data.drop(index=extreme_outliers.index)


In [16]:
# 10. Eliminar duplicados
data = data.drop_duplicates()


In [17]:
# 11. Eliminar filas con "No disponible" en vivienda
data = data[~data["vivienda"].str.strip().str.lower().eq("no disponible")]


In [24]:
# 11.5 Calcular precio promedio por m2 de propiedades cercanas
from sklearn.neighbors import BallTree
import numpy as np

# Primero, crear una copia de los datos sin valores nulos en lat/long para el cálculo
data_valid_coords = data.dropna(subset=['latitud', 'longitud']).copy()

# Convertir lat/long a radianes para el cálculo de distancia
data_valid_coords['lat_rad'] = np.radians(data_valid_coords['latitud'])
data_valid_coords['long_rad'] = np.radians(data_valid_coords['longitud'])

# Crear el árbol de búsqueda solo con coordenadas válidas
coords = np.column_stack([data_valid_coords['lat_rad'], data_valid_coords['long_rad']])
tree = BallTree(coords, metric='haversine')

# Calcular precio por m2 para cada propiedad
data_valid_coords['price_per_m2'] = data_valid_coords['price'] / data_valid_coords['m2_totales']

# Encontrar vecinos cercanos (dentro de 1km) y calcular precio promedio
RADIUS_KM = 1.0  # Radio de búsqueda en kilómetros
earth_radius = 6371  # Radio de la tierra en km

# Función para calcular el precio promedio de propiedades cercanas
def get_nearby_avg_price(idx):
    # Obtener coordenadas de la propiedad actual
    lat, lon = data_valid_coords.iloc[idx][['lat_rad', 'long_rad']]

    # Buscar propiedades dentro del radio
    indices = tree.query_radius([[lat, lon]], r=RADIUS_KM/earth_radius)[0]

    # Excluir la propiedad actual
    indices = indices[indices != idx]

    if len(indices) > 0:
        # Tomar hasta 10 propiedades más cercanas
        indices = indices[:10]
        # Calcular precio promedio por m2
        return data_valid_coords.iloc[indices]['price_per_m2'].mean()
    return None

# Aplicar la función a cada propiedad con coordenadas válidas
data_valid_coords['precio_m2_cercano'] = [get_nearby_avg_price(i) for i in range(len(data_valid_coords))]

# Crear la columna en el dataset original y asignar los valores calculados
data['precio_m2_cercano'] = np.nan
data.loc[data_valid_coords.index, 'precio_m2_cercano'] = data_valid_coords['precio_m2_cercano']

# Eliminar columnas temporales
data = data.drop(['lat_rad', 'long_rad', 'price_per_m2'], axis=1, errors='ignore')

In [22]:
# 11.6 Agregar precios promedio por barrio
# Cargar y limpiar datos de precios por barrio
precios_barrios = pd.read_csv('precios_bsas.csv', encoding='latin1')
precios_barrios.columns = ['barrio', 'precio_m2_barrio']
# Reemplazar valores no válidos con NaN
precios_barrios['precio_m2_barrio'] = precios_barrios['precio_m2_barrio'].replace(['///', '#N/A'], np.nan)
precios_barrios['precio_m2_barrio'] = precios_barrios['precio_m2_barrio'].str.replace('.', '').str.replace(',', '.').astype(float)

# Crear un diccionario de precios por barrio
precios_dict = dict(zip(precios_barrios['barrio'], precios_barrios['precio_m2_barrio']))

# Función para extraer el barrio de la ubicación
def extract_barrio(location):
    if pd.isna(location):
        return None
    # Lista de barrios para buscar
    barrios = list(precios_dict.keys())
    for barrio in barrios:
        if barrio.lower() in location.lower():
            return barrio
    return None

# Extraer barrio y asignar precio promedio usando data_old que contiene la columna 'location'
data['barrio'] = data_old['location'].apply(extract_barrio)
data['precio_m2_barrio'] = data['barrio'].map(precios_dict)

# Estandarizar los precios por barrio para que estén en la misma escala que los otros precios
data['precio_m2_barrio'] = data['precio_m2_barrio'] * 1000  # Convertir a USD aproximadamente

# Eliminar la columna temporal de barrio
data = data.drop('barrio', axis=1)

In [25]:
# 12. Imputar valores nulos
columnas_media_round = ['antiguedad', 'dormitorios', 'baños', 'ambientes']
columnas_media = ['m2_totales', 'm2_cubiertos','expenses','latitud', 'longitud', 'precio_m2_cercano', 'precio_m2_barrio']

mask_casas = data['vivienda'].str.contains('Casa', case=False, na=False)
data.loc[mask_casas, 'expenses'] = data.loc[mask_casas, 'expenses'].fillna(0)
for columna in columnas_media_round:
    data[columna] = data.groupby('vivienda')[columna].transform(lambda x: x.fillna(round(x.mean(), 0)))
for columna in columnas_media:
    data[columna] = data.groupby('vivienda')[columna].transform(lambda x: x.fillna(x.mean()))
data['cocheras'] = data['cocheras'].fillna(0)
data = data.dropna(subset=['price'])


In [26]:
# 13. Estandarizar variables numéricas, menos precio porque es la variable objetivo y es logarítmica
columnas_salida = ['Cantidad_plantas', 'antiguedad', 'dormitorios', 'cocheras', 'baños',
                   'ambientes', 'm2_totales', 'm2_cubiertos', 'expenses','latitud', 'longitud','precio_m2_cercano','precio_m2_barrio']
scaler = StandardScaler()
scaled_values = scaler.fit_transform(data[columnas_salida])
df_scaled = pd.DataFrame(scaled_values, columns=columnas_salida, index=data.index)
data[columnas_salida] = df_scaled

#Logaritmo en la variable objetivo 'price'
data['price'] = np.log1p(data['price'])


In [27]:
# 14. Codificar variables categóricas
columnas_categoricas = ["Pileta","Apto_credito","vivienda"]
onehot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop="first")
encoded_data = onehot_encoder.fit_transform(data[columnas_categoricas])
feature_names = onehot_encoder.get_feature_names_out(columnas_categoricas)
encoded_df = pd.DataFrame(encoded_data, columns=feature_names, index=data.index)
data = data.drop(columns=columnas_categoricas)
data = pd.concat([data, encoded_df], axis=1)


In [33]:
data

,Cantidad_plantas,antiguedad,dormitorios,cocheras,baños,ambientes,m2_totales,m2_cubiertos,expenses,price,...,longitud,nearby_avg_price_m2,precio_m2_barrio,precio_m2_cercano,Pileta_sí,Apto_credito_sí,vivienda_Departamento,vivienda_PH,original_lat,original_lon
0,-0.117407,-0.135918,1.289863,-0.050272,1.121699,0.792867,0.516783,1.969679,-0.715738,13.287880,...,0.184388,1686.567164,-0.563223,0.088835,0.0,0.0,0.0,0.0,-34.400390,-58.653931
2,-0.117407,1.071301,0.331645,-0.050272,-1.003487,0.143647,-0.785713,-0.728626,0.411538,11.931642,...,0.208896,NaN,0.833456,0.791214,0.0,0.0,1.0,0.0,-34.492504,-58.528998
4,8.517362,-0.890430,0.331645,-0.050272,1.121699,0.143647,3.371502,0.932830,1.936677,12.847929,...,0.157092,415.036630,-0.563223,-1.633849,1.0,0.0,0.0,0.0,-34.349050,-58.793080
5,-0.117407,-0.940731,2.248080,0.309643,5.372073,2.091308,1.212101,3.556183,-0.715738,13.038767,...,-0.864337,1257.227049,-0.563223,-0.233340,0.0,0.0,0.0,0.0,-34.000000,-64.000000
9,-0.117407,1.574310,0.331645,-0.050272,0.059106,0.143647,-0.462538,0.145825,0.942021,12.100718,...,-0.899854,719.496229,0.833456,-1.221364,0.0,0.0,1.0,0.0,-31.413500,-64.181050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1169,-0.117407,0.316789,0.331645,-0.410188,0.059106,0.792867,0.301332,0.220778,-0.715738,11.849405,...,0.173815,973.606382,-0.563223,-0.877093,0.0,0.0,0.0,0.0,-34.623511,-58.707828
1170,-0.117407,0.568293,0.331645,0.309643,-1.003487,0.143647,0.448230,0.220778,-0.715738,11.775297,...,-0.864337,1257.227049,-0.563223,-0.330936,0.0,0.0,0.0,0.0,-34.000000,-64.000000
1171,-0.117407,0.065285,0.331645,0.309643,0.059106,0.792867,0.448230,-0.528752,-0.715738,11.849405,...,-0.864337,1257.227049,-0.563223,-0.330936,0.0,0.0,0.0,0.0,-34.000000,-64.000000
1172,-0.117407,-0.739528,0.331645,0.309643,-1.003487,-1.154794,0.203400,-0.528752,-0.715738,11.759793,...,0.183519,1093.216260,-0.563223,-0.315002,0.0,0.0,0.0,0.0,-34.655820,-58.658360


In [34]:
# 15. Guardar el dataset final y los objetos de transformación
# Add original coordinates before saving
data['original_lat'] = data_old['latitud']
data['original_lon'] = data_old['longitud']

data.to_csv("dataset_final.csv", index=False)
joblib.dump(scaler, 'models/standardscaler.joblib')
joblib.dump(onehot_encoder, 'models/onehotencoder.joblib')
print("Preprocesamiento finalizado y archivos guardados.")


Preprocesamiento finalizado y archivos guardados.
